In [ ]:
%%html
<style>
  table { float: left; background: #d9d9d9; }
</style>

# find_vm_by_nameメソッド

```python
def find_vm_by_name(content, vm_name, folder=None, recurse=True):
```

vm_nameを指定してManaged Objectを検索するメソッドです。  

## 引数

| args    | type                     | param                                          |
| :-      | :----------------------- | :--------------------------------              |
| content | class                    | ServiceContent Object                          |
| vm_name | str                      | 検索するVM名                                   |
| folder  | class                    | VMを検索する基準となるフォルダのManaged Object |
| recurse | bool                     | 再帰的に検索するかどうかのフラグ               |

## 戻り値

| rtype    | return             |
| :------- | :--------          |
| class    | VMのManaged Object |

# find_vm_by_nameメソッド動作確認

find_vm_by_nameメソッドの動作確認をしてみます。  
以下の変数は環境に合わせて変更してください。

* vm_name
* folder_name
* hostname
* username
* password
* validate_certs

`folder_name` を指定した場合は、指定したフォルダーを基準に `vm_name` のVMを検索する処理も入れてみます。

In [ ]:
from ansible.module_utils.vmware import connect_to_api, find_obj, find_vm_by_name
from pyVmomi import vim

vm_name = 'VM Name'
folder_name = 'Folder Name'

class module():
    def __init__(self):
        self.params = {}

m = module()
m.params['hostname'] = 'vCenter IP or HostName'
m.params['username'] = 'UserName'
m.params['password'] = 'Password'
m.params['validate_certs'] = False

sc = connect_to_api(m)
vm_obj = find_vm_by_name(sc, vm_name)
if(vm_obj):
    print(vm_obj.name)
else:
    print("%s not found" % vm_name)

if(folder_name):
    folder_obj = find_obj(sc, [vim.Folder], folder_name)
    if(folder_obj):
        vm_obj = find_vm_by_name(sc, vm_name, folder=folder_obj)
        print(vm_obj.name)
    else:
        print("%s not found" % folder_name)

# find_vm_by_nameメソッドを使ったモジュール例

find_vm_by_nameを使って実際にAnsibleモジュールを作ってみましょう。  
以下のモジュールはVM名を指定してフィルタリングされた結果を戻すだけの単純なものです。  
フォルダも指定できるようにします。ただし、必須ではありません。  
モジュールは `/tmp/ansible_salf_made_module` に `get_vm_name.py` で保存されます。  
`/tmp/ansible_salf_made_module` ディレクトリが無い場合は作成してください。  
既にディレクトリがある場合は、以下を実行してモジュールファイルを作成します。

In [ ]:
%%writefile /tmp/ansible_salf_made_module/get_vm_name.py
#!/usr/bin/python

try:
    from pyVmomi import vim, vmodl
    HAS_PYVMOMI = True
except ImportError:
    HAS_PYVMOMI = False

from ansible.module_utils.vmware import find_obj, find_vm_by_name, connect_to_api, vmware_argument_spec
from ansible.module_utils.basic import AnsibleModule

def main():
    argument_spec = vmware_argument_spec()
    argument_spec.update(name=dict(type='str', required=True),
                        folder=dict(type='str'))

    module = AnsibleModule(argument_spec=argument_spec, supports_check_mode=True)

    if not HAS_PYVMOMI:
        module.fail_json(msg='pyvmomi python library not found')

    vm_name = module.params['name']
    folder_name = module.params['folder']

    result = dict(changed=False)
    sc = connect_to_api(module)
    
    if(folder_name):
        folder_obj = find_obj(sc, [vim.Folder], folder_name)
        if(folder_obj):
            vm_obj = find_vm_by_name(sc, vm_name, folder=folder_obj)
            if(vm_obj):
                result['vm'] = vm_obj.name
        else:
            module.fail_json(msg="%s not found" % folder_name)
    else:
        vm_obj = find_vm_by_name(sc, vm_name)
        if(vm_obj):
            result['vm'] = vm_obj.name
        else:
            module.fail_json(msg="%s not found" % vm_name)

    module.exit_json(**result)

if __name__ == "__main__":
    main()

# モジュールを動かす

作ったモジュールを実際に動かしてみます。  
以下のオプションは環境に合わせて変更してください。

* hostname
* username
* password
* validate_certs
* name

変更後、Kernelを `Ansible` に変更して以下を実行します。

In [ ]:
#ansible.cfg
[defaults]
library = /tmp/ansible_salf_made_module

In [ ]:
#inventory
[all]
localhost

In [ ]:
#play
name: salf-made module test
hosts: localhost
connection: local
gather_facts: no

In [ ]:
#task
get_vm_name:
  hostname: vCenter IP or HostName
  username: UserName
  password: Password
  validate_certs: no
  name: VM Name
  #folder: Folder Name

`name` に指定したVM名が取得できていることを確認します。  
フォルダを指定した時もVM名が取得できていることを確認します。